In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd "/content/drive/MyDrive/Nego_UBAR/CLB"

/content/drive/MyDrive/Nego_UBAR/CLB


In [ ]:
ls

annotated_user_da_with_span_full.json  clb-processed/
clb-analysis/                          CraigListBergain.csv
CLB_GAT_act.csv                        data.json
CLB_GAT_both.csv                       data.json.zip
CLB_GAT_int_act.csv                    glove.6B.300d.txt
CLB_GAT_int_both.csv                   mapping.pair
CLB_GAT_int_n_strategy.csv             testListFile.json
CLB_GAT_n_strategy.csv                 turn_lengths.txt
CLB_GAT_single_act.csv                 valListFile.json
CLB_gnn.csv


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy.stats import kde

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
clb = pd.read_csv("CraigListBergain.csv")
clb.head(10)

,User,Agent,Slot-Value,Value,Negotiation Strategy User,Negotiation Strategy Agent,Agent Dialogue Act,Intent
0,1,1,1,1,1,1,1,1
1,hi not sure if the charger would work for my car . can you sell it to me for 5 ?,it will work i have never seen a car without a cigarette lighter port .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer', 'politeness_buyer_greet']","['first_person_singular_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'personal_concern_seller', 'liwc_certainty']",unknown,init-price
2,still can i buy it for 5 ? i m on a tight budge,i think the lowest i would want to go is 8 .,NaN,NaN,"['first_person_singular_count_buyer', 'third_person_singular_buyer']","['first_person_singular_count_seller', 'number_of_diff_dic_pos', 'seller_propose', 'hedge_count_seller', 'assertive_count_seller']",counter-price,insist
3,how about 6 and i pick it up myself ? it ll save you shipping to me .,7 and we have a deal .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer']","['first_person_plural_count_seller', 'number_of_diff_dic_pos']",counter-price,counter-price
4,eh fine . 7 .,NaN,NaN,NaN,"['buyer_pos_sentiment', 'buyer_propose']",[''],NaN,agree
5,2,2,2,2,2,2,2,2
6,hi i am interested in your board !,hi do you have any questions ?,NaN,NaN,"['first_person_singular_count_buyer', 'politeness_buyer_greet']","['number_of_diff_dic_pos', 'politeness_seller_greet']",unknown,intro
7,do you know what wood it is made out of ?,i do not know specifically but the brand is a gravity skateboard .,NaN,NaN,"['third_person_singular_buyer', 'factive_count_buyer']","['first_person_singular_count_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'factive_count_seller']",disagree,inquiry
8,the wheels seem nice on it but they could be a beter quality . would you accept <price>_-0.2 for the board ?,no that offer is too low . the board is pretty much brand new as it s been ridden only or times . the bone bearings are brand new .,Price_ratio=-0.2,-0.2,"['buyer_pos_sentiment', 'third_person_singular_buyer', 'third_person_plural_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer']","['seller_neg_sentiment', 'seller_pos_sentiment', 'third_person_singular_seller', 'number_of_diff_dic_pos']",vague-price,init-price
9,if i picked it up at your location would you accpet <price>_0.0 ?,make it <price>_0.4 and we got a deal .,Price_ratio=0.0,0.0,"['first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer']","['first_person_plural_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'seller_propose']",counter-price,counter-price


## Nego Strategy :::

In [ ]:
ns_agent_l = []
for i in range(len(clb)):
    if str(clb.iloc[i,5])!='nan' and len(str(clb.iloc[i,5]))>2 :
        s = clb.iloc[i,5][1:-1].split(',')
        try :
            for j in s:             
                ns_agent_l.append(j.split("\'")[1])
        except:
          ns_agent_l.append("N")
    else:
        ns_agent_l.append("N")
  
print(len(ns_agent_l))
print(len(list(set(ns_agent_l))))

109959
25


In [ ]:
ns_agent_l = set(ns_agent_l)
ns_agent_l.discard('')
ns_agent_l.discard('N')
ns_agent_l = list(ns_agent_l)
ns_agent_l

['factive_count_seller',
 'politeness_seller_please_s',
 'seller_propose',
 'politeness_seller_greet',
 'personal_concern_seller',
 'seller_neg_sentiment',
 'friend',
 'number_of_diff_dic_pos',
 'hedge_count_seller',
 'family',
 'politeness_seller_gratitude',
 'seller_pos_sentiment',
 'seller_trade_in',
 'who_propose',
 'third_person_singular_seller',
 'liwc_informal',
 'liwc_certainty',
 'third_person_plural_seller',
 'first_person_singular_count_seller',
 'politeness_seller_please',
 'assertive_count_seller',
 'first_person_plural_count_seller',
 'number_of_diff_dic_neg']

In [ ]:
ns_agent_l = list(set(ns_agent_l))
ind = list(np.arange(len(ns_agent_l)))

d_ns_agent = {}

for i in range(len(ind)):  
    d_ns_agent[ns_agent_l[i]] = ind[i]

d_ns_agent

{'factive_count_seller': 0,
 'politeness_seller_please_s': 1,
 'seller_propose': 2,
 'politeness_seller_greet': 3,
 'personal_concern_seller': 4,
 'seller_neg_sentiment': 5,
 'friend': 6,
 'number_of_diff_dic_pos': 7,
 'hedge_count_seller': 8,
 'family': 9,
 'politeness_seller_gratitude': 10,
 'seller_pos_sentiment': 11,
 'seller_trade_in': 12,
 'who_propose': 13,
 'third_person_singular_seller': 14,
 'liwc_informal': 15,
 'liwc_certainty': 16,
 'third_person_plural_seller': 17,
 'first_person_singular_count_seller': 18,
 'politeness_seller_please': 19,
 'assertive_count_seller': 20,
 'first_person_plural_count_seller': 21,
 'number_of_diff_dic_neg': 22}

In [ ]:
edge_tuple = []

for i in range(len(clb)):
    if not str(clb.iloc[i,0]).isdigit():
        s = clb.iloc[i,5][1:-1].split(",")
        if len(s) > 1 :
            a = [j.split("\'")[1] for j in s]
            
            if "" in a:
                a.remove("")
            if "N" in a:
                a.remove("N")    

            x = a[:-1]
            y = a[1:]

            x = [[d_ns_agent[j]] for j in x]
            y = [[d_ns_agent[j]] for j in y]

            edge = [(x[i][0],y[i][0]) for i in range(len(x))]
            edge_tuple = edge_tuple + edge

In [ ]:
edge_tuple = list(set(edge_tuple))
len(edge_tuple)

264

In [ ]:
edge_source = [i[0] for i in edge_tuple]
edge_dest = [i[1] for i in edge_tuple]

edge_index = [edge_source,edge_dest]

In [ ]:
x

[[7]]

In [ ]:
x = [[i] for i in range(len(ns_agent_l))]
x = torch.tensor(x, dtype=torch.float)
edge_index = torch.tensor(edge_index, dtype=torch.long)
dataset = Data(x=x, y=y, edge_index=edge_index)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset.to(device)


<ipython-input-17-84d166f974b6>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype=torch.long)


In [ ]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        self.num_classes = 25
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, edge_dim = 1)
        self.conv2 = GATConv(self.hid*self.in_head, 50, concat=False,
                             heads=8, edge_dim = 1)
        self.conv3 = GATConv(50,25,concat=False, edge_dim = 1)
        self.conv4 = GATConv(25,5)

    def forward(self, data):
        x, edge_index= data.x, data.edge_index
        x = (self.conv1(x, edge_index))
        x = (self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)
        
        return (x)
        
model = GAT().to(device)

In [ ]:
model.train()
outputs = []



for i in range(50):   
    out = model(data).type(torch.FloatTensor)
    outputs.append(out)

In [ ]:
outputs[-1]*10000

tensor([[-3678.2053,  1618.5809, -5768.2031,  5368.6572,  7347.3550],
        [-3674.6389,  1610.1393, -5794.4219,  5390.0767,  7376.8257],
        [-3687.0627,  1631.0425, -5769.0791,  5368.8389,  7348.3311],
        [-3675.8547,  1609.2190, -5803.3364,  5398.7119,  7388.0952],
        [-3673.7910,  1610.5140, -5781.2266,  5380.2495,  7361.6938],
        [-3676.0610,  1611.1135, -5808.4692,  5402.5464,  7392.7002],
        [-3671.6311,  1605.2032, -5797.6538,  5393.4683,  7381.1807],
        [-3677.8882,  1616.1155, -5790.3965,  5387.6812,  7371.7378],
        [-3685.0459,  1624.9315, -5776.1484,  5377.4775,  7357.2119],
        [-3672.4187,  1607.6600, -5802.8262,  5395.9404,  7385.0557],
        [-3670.8674,  1605.6176, -5796.2329,  5389.9639,  7378.4043],
        [-3680.1731,  1617.6873, -5799.5718,  5395.7485,  7382.2944],
        [-3666.8896,  1604.1775, -5774.7920,  5370.6484,  7353.0776],
        [-3665.2256,  1598.4028, -5773.7588,  5374.2451,  7354.6411],
        [-3683.9568,

In [ ]:
n_strategy_dict = {'assertive_count_seller': 0,
 'seller_neg_sentiment': 1,
 'politeness_seller_please': 2,
 'first_person_plural_count_seller': 3,
 'third_person_singular_seller': 4,
 'politeness_seller_greet': 5,
 'third_person_plural_seller': 6,
 'liwc_certainty': 7,
 'hedge_count_seller': 8,
 'first_person_singular_count_seller': 9,
 'who_propose': 10,
 'seller_propose': 11,
 'number_of_diff_dic_pos': 12,
 'politeness_seller_please_s': 13,
 'friend': 14,
 'seller_pos_sentiment': 15,
 'family': 16,
 'seller_trade_in': 17,
 'liwc_informal': 18,
 'personal_concern_seller': 19,
 'factive_count_seller': 20,
 'politeness_seller_gratitude': 21,
 'number_of_diff_dic_neg': 22}

n_strategy_GAT_embeddings = [[-3678.2053,  1618.5809, -5768.2031,  5368.6572,  7347.3550],
        [-3674.6389,  1610.1393, -5794.4219,  5390.0767,  7376.8257],
        [-3687.0627,  1631.0425, -5769.0791,  5368.8389,  7348.3311],
        [-3675.8547,  1609.2190, -5803.3364,  5398.7119,  7388.0952],
        [-3673.7910,  1610.5140, -5781.2266,  5380.2495,  7361.6938],
        [-3676.0610,  1611.1135, -5808.4692,  5402.5464,  7392.7002],
        [-3671.6311,  1605.2032, -5797.6538,  5393.4683,  7381.1807],
        [-3677.8882,  1616.1155, -5790.3965,  5387.6812,  7371.7378],
        [-3685.0459,  1624.9315, -5776.1484,  5377.4775,  7357.2119],
        [-3672.4187,  1607.6600, -5802.8262,  5395.9404,  7385.0557],
        [-3670.8674,  1605.6176, -5796.2329,  5389.9639,  7378.4043],
        [-3680.1731,  1617.6873, -5799.5718,  5395.7485,  7382.2944],
        [-3666.8896,  1604.1775, -5774.7920,  5370.6484,  7353.0776],
        [-3665.2256,  1598.4028, -5773.7588,  5374.2451,  7354.6411],
        [-3683.9568,  1622.3293, -5793.1226,  5392.7603,  7375.5898],
        [-3671.1118,  1608.0133, -5793.6060,  5387.9678,  7374.5542],
        [-3671.3313,  1609.9087, -5785.5190,  5380.2681,  7365.0229],
        [-3695.2578,  1641.3744, -5784.1821,  5386.5093,  7363.1035],
        [-3680.8018,  1619.3429, -5798.4678,  5396.2827,  7379.9531],
        [-3668.8899,  1606.0234, -5797.3999,  5388.9507,  7377.0439],
        [-3681.2490,  1622.4958, -5767.4448,  5368.0068,  7346.4795],
        [-3693.9114,  1631.8101, -5792.3530,  5398.5034,  7378.8418],
        [-3688.4163,  1633.0482, -5776.4917,  5376.8892,  7355.5005]]

In [ ]:
clb['GAT_n_strategy_agent'] = np.zeros(len(clb))

for i in range(len(clb)):
    ns_agent_l = []
    if str(clb.iloc[i,5])!='nan' and len(str(clb.iloc[i,5]))>2 :
        s = clb.iloc[i,5][1:-1].split(',')
        try :
            for j in s:    
                a = j.split("\'")[1]         
                ns_agent_l.append(a)

                # for embedding size == 1 :
                # ns_agent_l.append(str(int(n_strategy_GAT_embeddings[n_strategy_dict[a]][0])))

                #for embedding size > 1 :
                l = n_strategy_GAT_embeddings[n_strategy_dict[a]]
                for ll in l :
                    ns_agent_l.append(str(int(ll)))
        except:
          ns_agent_l.append("")
    else:
        ns_agent_l.append("")

    clb['GAT_n_strategy_agent'][i] = str(ns_agent_l)

clb.head(3)

<ipython-input-51-2a6ca32d69b6>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clb['GAT_n_strategy_agent'][i] = str(ns_agent_l)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,User,Agent,Slot-Value,Value,Negotiation Strategy User,Negotiation Strategy Agent,Agent Dialogue Act,Intent,GAT_n_strategy_agent
0,1,1,1,1,1,1,1,1,['']
1,hi not sure if the charger would work for my car . can you sell it to me for 5 ?,it will work i have never seen a car without a cigarette lighter port .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer', 'politeness_buyer_greet']","['first_person_singular_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'personal_concern_seller', 'liwc_certainty']",unknown,init-price,"['first_person_singular_count_seller', '-3672', '1607', '-5802', '5395', '7385', 'third_person_singular_seller', '-3673', '1610', '-5781', '5380', '7361', 'number_of_diff_dic_pos', '-3666', '1604', '-5774', '5370', '7353', 'number_of_diff_dic_neg', '-3688', '1633', '-5776', '5376', '7355', 'personal_concern_seller', '-3668', '1606', '-5797', '5388', '7377', 'liwc_certainty', '-3677', '1616', '-5790', '5387', '7371']"
2,still can i buy it for 5 ? i m on a tight budge,i think the lowest i would want to go is 8 .,NaN,NaN,"['first_person_singular_count_buyer', 'third_person_singular_buyer']","['first_person_singular_count_seller', 'number_of_diff_dic_pos', 'seller_propose', 'hedge_count_seller', 'assertive_count_seller']",counter-price,insist,"['first_person_singular_count_seller', '-3672', '1607', '-5802', '5395', '7385', 'number_of_diff_dic_pos', '-3666', '1604', '-5774', '5370', '7353', 'seller_propose', '-3680', '1617', '-5799', '5395', '7382', 'hedge_count_seller', '-3685', '1624', '-5776', '5377', '7357', 'assertive_count_seller', '-3678', '1618', '-5768', '5368', '7347']"


In [ ]:
clb['GAT_n_strategy_agent'][1]

"['first_person_singular_count_seller', '-3672', '1607', '-5802', '5395', '7385', 'third_person_singular_seller', '-3673', '1610', '-5781', '5380', '7361', 'number_of_diff_dic_pos', '-3666', '1604', '-5774', '5370', '7353', 'number_of_diff_dic_neg', '-3688', '1633', '-5776', '5376', '7355', 'personal_concern_seller', '-3668', '1606', '-5797', '5388', '7377', 'liwc_certainty', '-3677', '1616', '-5790', '5387', '7371']"

In [ ]:
clb.to_csv("CLB_GAT_5_emb_int_n_strategy.csv", index=False)

## Action :::

In [ ]:
clb = pd.read_csv("CLB_GAT_act.csv")
clb.head(2)

,User,Agent,Slot-Value,Value,Negotiation Strategy User,Negotiation Strategy Agent,Agent Dialogue Act,Intent,GAT_act_agent
0,1,1,1,1,1,1,1,1,['']
1,hi not sure if the charger would work for my car . can you sell it to me for 5 ?,it will work i have never seen a car without a cigarette lighter port .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer', 'politeness_buyer_greet']","['first_person_singular_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'personal_concern_seller', 'liwc_certainty']",unknown,init-price,"['unknown', '10641.959']"


In [ ]:
act_agent_l = []
for i in range(len(clb)):
    s = clb.iloc[i,6]
    if str(s)!='nan' and not str(s).isdigit() :       
        try :   
            for j in s.split(","):      
                act_agent_l.append(j)
        except:
            act_agent_l.append("N")
    else:
        act_agent_l.append("N")
  
print(len(act_agent_l))
print(len(list(set(act_agent_l))))

32836
13


In [ ]:
act_agent_l = list(set(act_agent_l))
act_agent_l

['agree',
 'vague-price',
 'N',
 'intro',
 'unknown',
 'disagreedisagree',
 'inform',
 'init-price',
 'counter-pricedisagree',
 'insist',
 'counter-price',
 'disagree',
 'inquiry']

In [ ]:
act_agent_l = set(act_agent_l)
# act_agent_l.discard('')
act_agent_l.discard('N')
act_agent_l = list(act_agent_l)
act_agent_l

['disagree',
 'agree',
 'vague-price',
 'intro',
 'unknown',
 'disagreedisagree',
 'init-price',
 'counter-pricedisagree',
 'insist',
 'counter-price',
 'inform',
 'inquiry']

In [ ]:
act_agent_l = list(set(act_agent_l))
ind = list(np.arange(len(act_agent_l)))

d_act_agent = {}

for i in range(len(ind)):  
    d_act_agent[act_agent_l[i]] = ind[i]

d_act_agent

{'agree': 0,
 'vague-price': 1,
 'intro': 2,
 'unknown': 3,
 'disagreedisagree': 4,
 'init-price': 5,
 'inform': 6,
 'counter-pricedisagree': 7,
 'insist': 8,
 'counter-price': 9,
 'disagree': 10,
 'inquiry': 11}

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        self.num_classes = 25
        
        self.conv1 = GATConv(1, self.hid, heads=self.in_head, edge_dim = 1)
        self.conv2 = GATConv(self.hid*self.in_head, 50, concat=False,
                             heads=8, edge_dim = 1)
        self.conv3 = GATConv(50,1,concat=True, edge_dim = 1)

    def forward(self, data):
        x, edge_index= data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        
        return F.selu(x)
        
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT().to(device)

In [ ]:
edge_tuple = []

for i in range(len(clb)):
    if not str(clb.iloc[i,0]).isdigit():
        s = str(clb.iloc[i,6]).split(",")
        if len(s) > 1 :
            a = s
                        
            if "" in a:
                a.remove("")
            if "N" in a:
                a.remove("N")    

            x = a[:-1]
            y = a[1:]

            x = [[d_act_agent[j]] for j in x]
            y = [[d_act_agent[j]] for j in y]

            edge = [(x[i][0],y[i][0]) for i in range(len(x))]
            edge_tuple = edge_tuple + edge

In [ ]:
edge_tuple = list(set(edge_tuple))
len(edge_tuple)

42

In [ ]:
edge_source = [i[0] for i in edge_tuple]
edge_dest = [i[1] for i in edge_tuple]

edge_index = [edge_source,edge_dest]

In [ ]:
x

[[2]]

In [ ]:
model.train()
outputs = []

x = [[i] for i in range(len(act_agent_l))]
x = torch.tensor(x, dtype=torch.float)
edge_index = torch.tensor(edge_index, dtype=torch.long)
dataset = Data(x=x, y=y, edge_index=edge_index)
data = dataset.to(device)


for i in range(50):   
    out = model(data).type(torch.FloatTensor)
    outputs.append(out)

<ipython-input-490-004956f59131>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype=torch.long)


In [ ]:
outputs[0]*100000

tensor([[10337.1543],
        [11375.1621],
        [ 4848.9819],
        [10641.9590],
        [ 9697.9639],
        [ 9801.6201],
        [11635.8428],
        [16971.4336],
        [11822.1680],
        [ 9800.2979],
        [ 9909.3330],
        [10277.1064]], grad_fn=<MulBackward0>)

In [ ]:
act_dict = {'agree': 0,
 'vague-price': 1,
 'intro': 2,
 'unknown': 3,
 'disagreedisagree': 4,
 'init-price': 5,
 'inform': 6,
 'counter-pricedisagree': 7,
 'insist': 8,
 'counter-price': 9,
 'disagree': 10,
 'inquiry': 11}

act_GAT_embeddings = [[10337.1543],
        [11375.1621],
        [ 4848.9819],
        [10641.9590],
        [ 9697.9639],
        [ 9801.6201],
        [11635.8428],
        [16971.4336],
        [11822.1680],
        [ 9800.2979],
        [ 9909.3330],
        [10277.1064]]



In [ ]:
clb['GAT_act_agent'] = np.zeros(len(clb))

for i in range(len(clb)):
    s = clb.iloc[i,6]
    act_agent_l = []
    if str(s)!='nan' and not str(s).isdigit() :       
        try :   
            for j in s.split(","):      
                act_agent_l.append(j)
                act_agent_l.append(str(int(act_GAT_embeddings[act_dict[j]][0])))
        except:
            act_agent_l.append("")
    else:
        act_agent_l.append("")

    clb['GAT_act_agent'][i] = str(act_agent_l)

clb.head(2)

<ipython-input-93-6d54ffdb43ad>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clb['GAT_act_agent'][i] = str(act_agent_l)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,User,Agent,Slot-Value,Value,Negotiation Strategy User,Negotiation Strategy Agent,Agent Dialogue Act,Intent,GAT_act_agent
0,1,1,1,1,1,1,1,1,['']
1,hi not sure if the charger would work for my car . can you sell it to me for 5 ?,it will work i have never seen a car without a cigarette lighter port .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer', 'politeness_buyer_greet']","['first_person_singular_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'personal_concern_seller', 'liwc_certainty']",unknown,init-price,"['unknown', '10641']"


In [ ]:
clb.to_csv("CLB_GAT_int_act.csv", index=False)

## Merging :::

In [ ]:
act_embed = pd.read_csv("CLB_GAT_int_act.csv")
act_embed.head(2)

,User,Agent,Slot-Value,Value,Negotiation Strategy User,Negotiation Strategy Agent,Agent Dialogue Act,Intent,GAT_act_agent
0,1,1,1,1,1,1,1,1,['']
1,hi not sure if the charger would work for my car . can you sell it to me for 5 ?,it will work i have never seen a car without a cigarette lighter port .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer', 'politeness_buyer_greet']","['first_person_singular_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'personal_concern_seller', 'liwc_certainty']",unknown,init-price,"['unknown', '10641']"


In [ ]:
n_strategy_embed = pd.read_csv("CLB_GAT_int_n_strategy.csv")
n_strategy_embed.head(2)

,User,Agent,Slot-Value,Value,Negotiation Strategy User,Negotiation Strategy Agent,Agent Dialogue Act,Intent,GAT_n_strategy_agent
0,1,1,1,1,1,1,1,1,['']
1,hi not sure if the charger would work for my car . can you sell it to me for 5 ?,it will work i have never seen a car without a cigarette lighter port .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer', 'politeness_buyer_greet']","['first_person_singular_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'personal_concern_seller', 'liwc_certainty']",unknown,init-price,"['first_person_singular_count_seller', '143', 'third_person_singular_seller', '164', 'number_of_diff_dic_pos', '184', 'number_of_diff_dic_neg', '19', 'personal_concern_seller', '169', 'liwc_certainty', '186']"


In [ ]:
n_strategy_embed['Agent Dialogue Act'] = act_embed['Agent Dialogue Act']
n_strategy_embed['GAT_act_agent'] = act_embed['GAT_act_agent']
n_strategy_embed.head(2)

,User,Agent,Slot-Value,Value,Negotiation Strategy User,Negotiation Strategy Agent,Agent Dialogue Act,Intent,GAT_n_strategy_agent,GAT_act_agent
0,1,1,1,1,1,1,1,1,[''],['']
1,hi not sure if the charger would work for my car . can you sell it to me for 5 ?,it will work i have never seen a car without a cigarette lighter port .,NaN,NaN,"['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'third_person_singular_buyer', 'buyer_propose', 'hedge_count_buyer', 'assertive_count_buyer', 'politeness_buyer_greet']","['first_person_singular_count_seller', 'third_person_singular_seller', 'number_of_diff_dic_pos', 'number_of_diff_dic_neg', 'personal_concern_seller', 'liwc_certainty']",unknown,init-price,"['first_person_singular_count_seller', '143', 'third_person_singular_seller', '164', 'number_of_diff_dic_pos', '184', 'number_of_diff_dic_neg', '19', 'personal_concern_seller', '169', 'liwc_certainty', '186']","['unknown', '10641']"


In [ ]:
n_strategy_embed.to_csv("CLB_GAT_int_both.csv", index=False)